#### Import RIM Cell, and LSTM cell (from tensorflow) as a comparison

In [40]:
from tfRIM import RIMCell
from tensorflow.keras.layers import LSTMCell
import tensorflow as tf
import numpy as np

In [41]:
timestamp = None
batch_size = None
input_size = 10

#### Replace the API for creating a LSTM Cell:
rnn_cell = LSTMCell(...)
#### by creating a RIM Cell:
rnn_cell = RIMCell(...)
##### We declare 20 hidden units per RIM; 6 RIMs and pick top 5 RIM to active; 2 input heads and communication heads; key/query/value sizes are 8 for both inputs and communications.

In [42]:
# rnn_cell = LSTMCell(100)
rnn_cell = RIMCell(units = 20, nRIM=6, k=5,
                num_input_heads=2, input_key_size=8, input_value_size=8, input_query_size=8, input_keep_prob=0.5,
                num_comm_heads=2, comm_key_size=8, comm_value_size=8, comm_query_size=8, comm_keep_prob=0.5)

#### Then build the neural net as usual

In [43]:
model = tf.keras.Sequential()
model.add(tf.keras.Input([timestamp, input_size], batch_size=batch_size))
model.add(tf.keras.layers.RNN(cell=rnn_cell, return_sequences = True, return_state=False))
model.add(tf.keras.layers.Dense(1))

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
cost = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt, loss=cost)

#### and fit/predict with data

In [44]:
data_x = np.random.rand(64, 100, 10) #batch_size = 64, timestamp=100, input_size=10
data_y = np.random.binomial(n=1, p=0.5, size = [64, 100, 1])

In [45]:
model.fit(data_x, data_y)

2/2 [==============================] - 0s 32ms/step - loss: 1.1293


In [46]:
pred_y = model.predict(data_x)

#### Performance is expected to be around 50% in this demonstration, since everything is purely random

In [47]:
np.mean((pred_y>0.5)==data_y)

0.501875